- 레스토랑 리뷰 감성 분류하기 :
https://github.com/rickiepark/nlp-with-pytorch/blob/main/chapter_3/3_5_Classifying_Yelp_Review_Sentiment.ipynb

- NLP using GloVe Embeddings(FAKE NEWS) : 
https://www.kaggle.com/code/madz2000/nlp-using-glove-embeddings-99-87-accuracy